<a href="https://colab.research.google.com/github/yej117/Cloud_Segmentation_Deep_Learning/blob/main/DL_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git
!pip install -U tfds-nightly

  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-xm17bzxt
  Running command git clone -q https://github.com/tensorflow/examples.git /tmp/pip-req-build-xm17bzxt
  Created wheel for tensorflow-examples: filename=tensorflow_examples-280abd2f5d901e02af8bd4300c699d0146749037_-cp36-none-any.whl size=188735 sha256=99a918f037e3f76475ef76f4c50d91850fbe9373c004316271b63f0f7c54960d
  Stored in directory: /tmp/pip-ephem-wheel-cache-7dz6y9h9/wheels/83/64/b3/4cfa02dc6f9d16bf7257892c6a7ec602cd7e0ff6ec4d7d714d
Successfully built tensorflow-examples
     |████████████████████████████████| 3.7MB 6.9MB/s 


In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds

from IPython.display import clear_output
import matplotlib.pyplot as plt

In [ ]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Dataset oxford_iiit_pet downloaded and prepared to /root/tensorflow_datasets/oxford_iiit_pet/3.2.0. Subsequent calls will reuse this data.


In [ ]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

In [ ]:
batch_size = 100
IMG_SHAPE = 128
#ImageDataGenerator generate batches of images with real-time augmentation
train_image_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
@tf.function
def load_image_train(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [ ]:
def load_image_test(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [ ]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)

In [ ]:
train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
for image, mask in train.take(1):
  sample_image, sample_mask = image, mask
display([sample_image, sample_mask])

In [ ]:
OUTPUT_CHANNELS = 3

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

In [ ]:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_dataset, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_dataset,
                          callbacks=[DisplayCallback()])

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(EPOCHS)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
show_predictions(test_dataset, 3)

In [ ]:
#From directory, change to read in from uploaded file

train_data_gen=train_image_gen.flow_from_directory(directory=train_dir,
                                               target_size=(IMG_SHAPE, IMG_SHAPE),
                                               batch_size=batch_size,
                                               shuffle=True,
                                               class_mode='categorical')
validation_image_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = validation_image_gen.flow_from_directory(directory=val_dir,
                                                               target_size=(IMG_SHAPE, IMG_SHAPE),
                                                               shuffle=True,
                                                               batch_size=batch_size,
                                                               class_mode='categorical')

image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               shuffle=True,
                                               directory=train_dir,
                                               target_s

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

#use ImageDataGenerator to create a transformation that rescales the 
#images by 255 and then applies a random 45 degree rotation. 
#Then use the .flow_from_directory method to apply the above 
#transformation to the images in our training set. 

image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45) 

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                           shuffle=True,
                           directory=train_dir,
                           target_size=(IMG_SHAPE, IMG_SHAPE))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
#use ImageDataGenerator to create a transformation that rescales the
# images by 255 and then applies a random zoom of up to 50%. 
#Then use the .flow_from_directory method to apply the above 
#transformation to the images in our training set.

image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5)

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               shuffle=True,
                                               directory=train_dir,
                                               target_size=(IMG_SHAPE, IMG_SHAPE))

augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

image_gen_train = ImageDataGenerator(rescale=1./255,
                                     rotation_range=45,
                                     zoom_range=0.5,
                                     horizontal_flip=True,
                                     width_shift_range=0.15,
                                     height_shift_range=0.15,
                                     shear_range=0.2,
                                     fill_mode='nearest')


train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE, IMG_SHAPE),
                                               directory=train_dir,
                                               class_mode='sparse')

augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

#Generally, we only apply data augmentation to our training examples. 
#So, in the cell below, use ImageDataGenerator to create a transformation
# that only rescales the images by 255. Then use the 
#.flow_from_directory method to apply the above transformation 
#to the images in our validation set.

image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 shuffle=True,
                                                 directory=val_dir,
                                                 target_size=(IMG_SHAPE,IMG_SHAPE))

model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(16, (3,3), activation='relu', 
                                                    input_shape=(150, 150,3)),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(5, activation='softmax')])
                             
model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

#train your model using the fit_generator function instead of 
#the usual fit function. We have to use the fit_generator function 
#because we are using the ImageDataGenerator class to generate 
#batches of training and validation data for our model. 
#Train the model for 80 epochs and make sure you use the 
#proper parameters in the fit_generator function

epochs = 10

history = model.fit_generator(
    train_data_gen,
    epochs = epochs) 

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize = (8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training accuracy')
plt.plot(epochs_range, val_acc, label='Validation accuracy')
plt.legend(loc='lower right')
plt.title("Training and validation accuracy")


plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training loss')
plt.plot(epochs_range, val_loss, label='Validation loss')
plt.legend(loc='upper right')
plt.title("Training and validation loss")
plt.savefig('./foo.png')
plt.show()

In [ ]:
DATA PREPARATION
#BY Using the image data generator class:
#Images must be formatted into appropriately pre-processed floating point
#tensors before being fed into the network
#Steps - 1) read images from disk -> 2) Decode contents of these images
# and convert it into proper grid format as per their RBG contnet
#->3) convert them into floating point tensors -> 4) Rescale the tensors
#from values between 0 and 255 to values between 0 and 1 as neural networks
#prefer working with smaller inputs

#All done by the tensorflow.keras.preprocessing.image.ImageDataGenerator
#Generator for training set
train_image_generator = ImageDataGenerator(rescale=1./255)
#Generator for validation set
validation_image_generator=ImageDataGenerator(rescale=1./255)

#After specifying our generators for training and validation, flow_from_
#directory method will load images from disk, apply rescaling and
#and resize 
#(Specify how training and validation data should read from disk):
train_data_ger = train_image_generator.flow_from_directory(
        batch_size = BATCH_SIZE,
        directory=train_dir,
        shuffle=True,
        target_size=(IMG_SHAPE, IMG_SHAPE), #resize to a 150x150 pixels
        class_mode='binary') #binary as only 2 classes: dog & cat
#Batch_size: read 100 images at a time, directory=where to read images from
#shuffling (to prevent memorising data by order) not needed for validation
val_data_ger = validation_image_generator.flow_from_directory(
        batch_size = BATCH_SIZE,
        directory=validation_dir,
        shuffle=False,
        target_size=(IMG_SHAPE, IMG_SHAPE),
        class_mode='binary')


#VISUALISING
#Grab some images and plot them

sample_training_images, _ = next(train_data_ger)

#The below function plots images as grid form with:
#1 row and 5 columns where images are placed in each column
#The function returns a batch from a data set - one batch is a tuple
#of (many images, many labels) - for now discard labels to look at images

def PlotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()
    
PlotImages(sample_training_images[:5]) #plot images 0-4

#MODEL CREATION
#Define the model:
#The model consists of 4 convulation blocks with max pooling in each of
#them. We then have a fully connected layer with 512 units, with relu
#activation function. The model outputs probabilities outputs for two 
#classes -- dogs, cats --  by using softmax

model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', 
                               input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')])

#COMPILE THE MODEL
# as usual - adam will be used as optimizer. Since using softmax 
#see the associated loss function used 'sparse_....'. We want to look
        #at training and validation accuracy on each epoch
        #as we train our network, so we pass the metrics argument
        
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#Creating a model summary to look at layers of network:

model.summary()


#TRAIN THE MODEL
Epochs = 3 #they use 10, but takes too long
history = model.fit_generator(
        train_data_ger,
        steps_per_epoch=int(np.ceil(total_val/float(BATCH_SIZE))),
        epochs=Epochs,
        validation_data=val_data_ger,
        validation_steps=int(np.ceil(total_val/float(BATCH_SIZE))))
#one sees that validation dataset presents overfitting (about 77%), 
#so model learnt
#examples too closely

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(Epochs)

plt.figure(figsize = (8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training accuracy')
plt.plot(epochs_range, val_acc, label='Validation accuracy')
plt.legend(loc='lower right')
plt.title("Training and validation accuracy")


plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training loss')
plt.plot(epochs_range, val_loss, label='Validation loss')
plt.legend(loc='upper right')
plt.title("Training and validation loss")
plt.savefig('./foo.png')
plt.show()

In [ ]:
#DATA PREPARATION
#BY Using the image data generator class:
#Images must be formatted into appropriately pre-processed floating point
#tensors before being fed into the network
#Steps - 1) read images from disk -> 2) Decode contents of these images
# and convert it into proper grid format as per their RBG contnet
#->3) convert them into floating point tensors -> 4) Rescale the tensors
#from values between 0 and 255 to values between 0 and 1 as neural networks
#prefer working with smaller inputs

#All done by the tensorflow.keras.preprocessing.image.ImageDataGenerator
#Generator for training set
train_image_generator = ImageDataGenerator(rescale=1./255)
#Generator for validation set
validation_image_generator=ImageDataGenerator(rescale=1./255)

#After specifying our generators for training and validation, flow_from_
#directory method will load images from disk, apply rescaling and
#and resize 
#(Specify how training and validation data should read from disk):
train_data_ger = train_image_generator.flow_from_directory(
        batch_size = BATCH_SIZE,
        directory=train_dir,
        shuffle=True,
        target_size=(IMG_SHAPE, IMG_SHAPE), #resize to a 150x150 pixels
        class_mode='binary') #binary as only 2 classes: dog & cat
#Batch_size: read 100 images at a time, directory=where to read images from
#shuffling (to prevent memorising data by order) not needed for validation
val_data_ger = validation_image_generator.flow_from_directory(
        batch_size = BATCH_SIZE,
        directory=validation_dir,
        shuffle=False,
        target_size=(IMG_SHAPE, IMG_SHAPE),
        class_mode='binary')


#VISUALISING
#Grab some images and plot them

sample_training_images, _ = next(train_data_ger)

#The below function plots images as grid form with:
#1 row and 5 columns where images are placed in each column
#The function returns a batch from a data set - one batch is a tuple
#of (many images, many labels) - for now discard labels to look at images

def PlotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()
    
#IMAGE AUGMENTATION To Improve Model
#Data Input Pipeline: how to get data from disk to interface with our model
#Workflow: Examine the data, build input pipeline, build, train, test model
#improve model, repeat the process

#Overfitting: small number of training examples. Augment data to have enough
# number and variety of data. Data augmentation generates more training data
#from existing inputs. By transformation, model will never see same image
#twice. Exposes model to more aspects of data to generalize better.

#1-Flipping the Image - randomly apply horizontal flip (Transformation)
image_gen=ImageDataGenerator(rescale=1./255, horizontal_flip=True)
#Horizontal flip is the augmentation

train_data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE,IMG_SHAPE))

#To see transformation take a sample from data set and repeat it 5 times
#Augmenation randomly applied

augmented_image = [train_data_gen[0][0][0] for i in range (5)]
PlotImages(augmented_image)

#Rotating the Image (by 45 degrees):
image_gen=ImageDataGenerator(rescale=1./255, rotation_range=45)
train_data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE,IMG_SHAPE))

#To see transformation take a sample from data set and repeat it 5 times
#Augmenation randomly applied

augmented_image = [train_data_gen[0][0][0] for i in range (5)]
PlotImages(augmented_image)

#Zooming the image: (reminder resizing is to put images of different sizes
#to one common size)
image_gen=ImageDataGenerator(rescale=1./255, zoom_range=0.5)
train_data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE,IMG_SHAPE))

augmented_image = [train_data_gen[0][0][0] for i in range (5)]
PlotImages(augmented_image)

#COMBINING THESE OPERATIONS:
image_gen_train = ImageDataGenerator(
        rescale=1./255, 
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE,IMG_SHAPE),
                                               class_mode='binary')

augmented_image = [train_data_gen[0][0][0] for i in range (5)]
PlotImages(augmented_image)

#Creating the validation data generator
#keep images without augmentation -should reflect images we would see
#once model has been trained
image_gen_val=ImageDataGenerator(rescale=1./255, zoom_range=0.5)
val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=validation_dir,
                                                 target_size=(IMG_SHAPE,IMG_SHAPE),
                                                 class_mode='binary')

#MODEL CREATION (DEFINE THE MODEL) before dense layer apply dropout of 50%
#meaning 50% of values entering this layer will be assigned 0 (prevent 
#overfitting) - changing below to generate Fractal-Unet
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', 
                               input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')])
        
#COMPILE THE MODEL
# as usual - adam will be used as optimizer. Since using softmax 
#see the associated loss function used 'sparse_....'. We want to look
        #at training and validation accuracy on each epoch
        #as we train our network, so we pass the metrics argument
        
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#Creating a model summary to look at layers of network:

model.summary()


#TRAIN THE MODEL
Epochs = 10 
history = model.fit_generator(
        train_data_ger,
        steps_per_epoch=int(np.ceil(total_val/float(BATCH_SIZE))),
        epochs=Epochs,
        validation_data=val_data_ger,
        validation_steps=int(np.ceil(total_val/float(BATCH_SIZE))))
#one sees that validation dataset presents overfitting (about 77%), 
#so model learnt
#examples too closely

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(Epochs)

plt.figure(figsize = (8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training accuracy')
plt.plot(epochs_range, val_acc, label='Validation accuracy')
plt.legend(loc='lower right')
plt.title("Training and validation accuracy")


plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training loss')
plt.plot(epochs_range, val_loss, label='Validation loss')
plt.legend(loc='upper right')
plt.title("Training and validation loss")
plt.savefig('./foo.png')
plt.show()

#Early Stopping: In this method, we track the loss on the validation set 
#during the training phase and use it to determine when to stop training 
#such that the model is accurate but not overfitting. Early Stopping 
#to prevent overfitting and can also be used to help us compare 
#different models and choose the best one.

#Image Augmentation: Artificially boosting the number of images in our
# training set by applying random image transformations to 
#the existing images in the training set.

#Dropout: Removing a random selection of a fixed number of neurons 
#in a neural network during training.

In [ ]:
''' Steps: 1) Uploading & Pre-processing 2) Compiling models 3) Testing Accuracy 5) trying with Fractal-Unet

In [ ]:
#From deep-segmentation, github: https://github.com/danielelic/deep-segmentation/blob/master/train_fractal_unet.py

def merge(inputs, mode, concat_axis=-1):
    return concatenate(inputs, concat_axis)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#below are evaluation metrics: recall=no. of correct guesses; precision: 
#% of times when forecast was correct; f1=best precision, recall
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall))

img_rows = 128
img_cols = 128

smooth = 1.
epochs = 200    


def get_fractalunet(f=16):
    inputs = Input((img_rows, img_cols, 1))

    conv1 = Convolution2D(f, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Convolution2D(f, 3, 3, activation='relu', border_mode='same')(conv1)

    down1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = BatchNormalization()(down1)
    conv2 = Convolution2D(2 * f, 3, 3, activation='relu', border_mode='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Convolution2D(2 * f, 3, 3, activation='relu', border_mode='same')(conv2)

    down2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = BatchNormalization()(down2)
    conv3 = Convolution2D(4 * f, 3, 3, activation='relu', border_mode='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Convolution2D(4 * f, 3, 3, activation='relu', border_mode='same')(conv3)

    down3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = BatchNormalization()(down3)
    conv4 = Convolution2D(8 * f, 3, 3, activation='relu', border_mode='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Convolution2D(8 * f, 3, 3, activation='relu', border_mode='same')(conv4)

    down4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = BatchNormalization()(down4)
    conv5 = Convolution2D(16 * f, 3, 3, activation='relu', border_mode='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Convolution2D(16 * f, 3, 3, activation='relu', border_mode='same')(conv5)

    up1 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=3)

    conv6 = BatchNormalization()(up1)
    conv6 = Convolution2D(8 * f, 3, 3, activation='relu', border_mode='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Convolution2D(8 * f, 3, 3, activation='relu', border_mode='same')(conv6)

    up2 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=3)

    conv7 = BatchNormalization()(up2)
    conv7 = Convolution2D(4 * f, 3, 3, activation='relu', border_mode='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Convolution2D(4 * f, 3, 3, activation='relu', border_mode='same')(conv7)

    up3 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=3)

    conv8 = BatchNormalization()(up3)
    conv8 = Convolution2D(2 * f, 3, 3, activation='relu', border_mode='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Convolution2D(2 * f, 3, 3, activation='relu', border_mode='same')(conv8)

    up4 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=3)

    conv9 = BatchNormalization()(up4)
    conv9 = Convolution2D(f, 3, 3, activation='relu', border_mode='same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Convolution2D(f, 3, 3, activation='relu', border_mode='same')(conv9)

    # --- end first u block

    down1b = MaxPooling2D(pool_size=(2, 2))(conv9)
    down1b = merge([down1b, conv8], mode='concat', concat_axis=3)

    conv2b = BatchNormalization()(down1b)
    conv2b = Convolution2D(2 * f, 3, 3, activation='relu', border_mode='same')(conv2b)
    conv2b = BatchNormalization()(conv2b)
    conv2b = Convolution2D(2 * f, 3, 3, activation='relu', border_mode='same')(conv2b)

    down2b = MaxPooling2D(pool_size=(2, 2))(conv2b)
    down2b = merge([down2b, conv7], mode='concat', concat_axis=3)

    conv3b = BatchNormalization()(down2b)
    conv3b = Convolution2D(4 * f, 3, 3, activation='relu', border_mode='same')(conv3b)
    conv3b = BatchNormalization()(conv3b)
    conv3b = Convolution2D(4 * f, 3, 3, activation='relu', border_mode='same')(conv3b)

    down3b = MaxPooling2D(pool_size=(2, 2))(conv3b)
    down3b = merge([down3b, conv6], mode='concat', concat_axis=3)

    conv4b = BatchNormalization()(down3b)
    conv4b = Convolution2D(8 * f, 3, 3, activation='relu', border_mode='same')(conv4b)
    conv4b = BatchNormalization()(conv4b)
    conv4b = Convolution2D(8 * f, 3, 3, activation='relu', border_mode='same')(conv4b)

    down4b = MaxPooling2D(pool_size=(2, 2))(conv4b)
    down4b = merge([down4b, conv5], mode='concat', concat_axis=3)

    conv5b = BatchNormalization()(down4b)
    conv5b = Convolution2D(16 * f, 3, 3, activation='relu', border_mode='same')(conv5b)
    conv5b = BatchNormalization()(conv5b)
    conv5b = Convolution2D(16 * f, 3, 3, activation='relu', border_mode='same')(conv5b)

    up1b = merge([UpSampling2D(size=(2, 2))(conv5b), conv4b], mode='concat', concat_axis=3)

    conv6b = BatchNormalization()(up1b)
    conv6b = Convolution2D(8 * f, 3, 3, activation='relu', border_mode='same')(conv6b)
    conv6b = BatchNormalization()(conv6b)
    conv6b = Convolution2D(8 * f, 3, 3, activation='relu', border_mode='same')(conv6b)

    up2b = merge([UpSampling2D(size=(2, 2))(conv6b), conv3b], mode='concat', concat_axis=3)

    conv7b = BatchNormalization()(up2b)
    conv7b = Convolution2D(4 * f, 3, 3, activation='relu', border_mode='same')(conv7b)
    conv7b = BatchNormalization()(conv7b)
    conv7b = Convolution2D(4 * f, 3, 3, activation='relu', border_mode='same')(conv7b)

    up3b = merge([UpSampling2D(size=(2, 2))(conv7b), conv2b], mode='concat', concat_axis=3)

    conv8b = BatchNormalization()(up3b)
    conv8b = Convolution2D(2 * f, 3, 3, activation='relu', border_mode='same')(conv8b)
    conv8b = BatchNormalization()(conv8b)
    conv8b = Convolution2D(2 * f, 3, 3, activation='relu', border_mode='same')(conv8b)

    up4b = merge([UpSampling2D(size=(2, 2))(conv8b), conv9], mode='concat', concat_axis=3)

    conv9b = BatchNormalization()(up4b)
    conv9b = Convolution2D(f, 3, 3, activation='relu', border_mode='same')(conv9b)
    conv9b = BatchNormalization()(conv9b)
    conv9b = Convolution2D(f, 3, 3, activation='relu', border_mode='same')(conv9b)
    conv9b = BatchNormalization()(conv9b)

    outputs = Convolution2D(1, 1, 1, activation='hard_sigmoid', border_mode='same')(conv9b)

    net = Model(inputs=inputs, outputs=outputs)
    net.compile(loss=dice_coef_loss, optimizer='adam', metrics=[dice_coef, 'accuracy', precision, recall, f1score])

    net.summary()

    return net

In [ ]:
def train_and_predict(bit):
    print('-' * 30)
    print('Loading and train data (bit = ' + str(bit) + ') ...')
    print('-' * 30)
    imgs_bit_train, imgs_bit_mask_train, _ = load_train_data(bit)

    print(imgs_bit_train.shape[0], imgs_bit_mask_train.shape[0])

    imgs_bit_train = imgs_bit_train.astype('float32')
    mean = np.mean(imgs_bit_train)
    std = np.std(imgs_bit_train)

    imgs_bit_train -= mean
    imgs_bit_train /= std

    imgs_bit_mask_train = imgs_bit_mask_train.astype('float32')
    imgs_bit_mask_train /= 255.  # scale masks to [0, 1]

    print('-' * 30)
    print('Creating and compiling model (bit = ' + str(bit) + ') ...')
    print('-' * 30)
    model = get_fractalunet(f=16)

    csv_logger = CSVLogger('log_fractalunet_' + str(bit) + '.csv')
    model_checkpoint = ModelCheckpoint('weights_fractalunet_' + str(bit) + '.h5', monitor='val_loss',
                                       save_best_only=True)

    print('-' * 30)
    print('Fitting model (bit = ' + str(bit) + ') ...')
    print('-' * 30)

    model.fit(imgs_bit_train, imgs_bit_mask_train, batch_size=32, epochs=epochs, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[csv_logger, model_checkpoint])

    print('-' * 30)
    print('Loading and preprocessing test data (bit = ' + str(bit) + ') ...')
    print('-' * 30)

    imgs_bit_test, imgs_mask_test, imgs_bit_id_test = load_test_data(bit)

    imgs_bit_test = imgs_bit_test.astype('float32')
    imgs_bit_test -= mean
    imgs_bit_test /= std

    print('-' * 30)
    print('Loading saved weights...')
    print('-' * 30)
    model.load_weights('weights_fractalunet_' + str(bit) + '.h5')

    print('-' * 30)
    print('Predicting masks on test data (bit = ' + str(bit) + ') ...')
    print('-' * 30)
    imgs_mask_test = model.predict(imgs_bit_test, verbose=1)

    if bit == 8:
        print('-' * 30)
        print('Saving predicted masks to files...')
        print('-' * 30)
        pred_dir = 'preds_8'
        if not os.path.exists(pred_dir):
            os.mkdir(pred_dir)
        for image, image_id in zip(imgs_mask_test, imgs_bit_id_test):
            image = (image[:, :, 0] * 255.).astype(np.uint8)
            imsave(os.path.join(pred_dir, str(image_id).split('/')[-1] + '_pred_fractalunet.png'), image)

    elif bit == 16:
        print('-' * 30)
        print('Saving predicted masks to files...')
        print('-' * 30)
        pred_dir = 'preds_16'
        if not os.path.exists(pred_dir):
            os.mkdir(pred_dir)
        for image, image_id in zip(imgs_mask_test, imgs_bit_id_test):
            image = (image[:, :, 0] * 255.).astype(np.uint8)
            imsave(os.path.join(pred_dir, str(image_id).split('/')[-1] + '_pred_fractalunet.png'), image)


if __name__ == '__main__':
    train_and_predict(8)
    train_and_predict(16)